In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import urllib
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

<b>Data</b>

Newyork has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. https://geo.nyu.edu/catalog/nyu_2451_34572

From Foursquare Venues Categories - https://developer.foursquare.com/docs/resources/categories Sushi category Id - 4bf58dd8d48988d1d2941735

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [4]:

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods_dataframe = pd.DataFrame(columns=column_names)

for data in newyork_data['features']:
    borough=data['properties']['borough']
    neigh_name=data['properties']['name']
    
    neigh_latlon = data['geometry']['coordinates']
    neigh_lat = neigh_latlon[1]
    neigh_lon = neigh_latlon[0]
    
    neighborhoods_dataframe=neighborhoods_dataframe.append({'Borough':borough, 'Neighborhood':neigh_name, 'Latitude':neigh_lat, 'Longitude':neigh_lon},ignore_index=True)


manhattan_data = neighborhoods_dataframe[neighborhoods_dataframe['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [5]:
geolocator = Nominatim(user_agent="newyorkcity")
location = geolocator.geocode("new york city")
latitude = location.latitude
longitude = location.longitude
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
map_newyork

In [6]:
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'],manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
map_newyork

In [9]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'AKYIS02NFAXJE4QBRPAKTGSTAAQNULHPM01BYJPVQ4I3WPCJ'
CLIENT_SECRET = 'SG430LSYZEHE2EEPSBBZKZKWUPADIDNLPBY4OYMHFDKKINXS'
VERSION = '20180323'

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [10]:
newyork_venues_sushi = getNearbyVenues(names=neighborhoods_dataframe['Neighborhood'], latitudes=neighborhoods_dataframe['Latitude'], longitudes=neighborhoods_dataframe['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1d2941735')
newyork_venues_sushi.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Fieldston,40.895437,-73.905643,Asian Tokyo,40.890839,-73.898335,Sushi Restaurant
1,Fieldston,40.895437,-73.905643,Yokohama,40.887214,-73.904708,Sushi Restaurant
2,Riverdale,40.890834,-73.912585,Yokohama,40.887214,-73.904708,Sushi Restaurant
3,Riverdale,40.890834,-73.912585,Planet Tokyo,40.886158,-73.909615,Sushi Restaurant
4,Kingsbridge,40.881687,-73.902818,Yokohama,40.887214,-73.904708,Sushi Restaurant


In [11]:
newyork_venues_sushi.shape

(1763, 7)

In [12]:

def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [25]:
map_newyork_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, local, venue, venueCat in zip(newyork_venues_sushi['Venue Latitude'], newyork_venues_sushi['Venue Longitude'], newyork_venues_sushi['Neighborhood'], newyork_venues_sushi['Venue'], newyork_venues_sushi['Venue Category']):  
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7).add_to(map_newyork_sushi)
    # addToMap(newyork_venues_sushi, 'red', map_newyork_sushi)

map_newyork_sushi

In [24]:
manhattan_grouped = newyork_venues_sushi.groupby('Neighborhood').count()
manhattan_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Annadale,2,2,2,2,2,2
Arden Heights,2,2,2,2,2,2
Astoria,14,14,14,14,14,14
Astoria Heights,1,1,1,1,1,1
Auburndale,5,5,5,5,5,5
Bath Beach,10,10,10,10,10,10
Battery Park City,20,20,20,20,20,20
Bay Ridge,11,11,11,11,11,11
Bay Terrace,2,2,2,2,2,2


<b> Analysis of each neighborhood</b>

In [26]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_sushi[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_sushi['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,...,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Snack Place,Steakhouse,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant
0,Fieldston,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,Fieldston,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,Riverdale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,Riverdale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Kingsbridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [29]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,...,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Snack Place,Steakhouse,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant
0,Annadale,0.000000,0.0,0.000,0.00,0.0,0.000000,0.0,0.00000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000,0.000000,0.000000
1,Arden Heights,0.000000,0.0,0.000,0.00,0.0,0.000000,0.0,0.00000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000,0.000000,0.000000
2,Astoria,0.071429,0.0,0.000,0.00,0.0,0.000000,0.0,0.00000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.857143,0.000,0.000000,0.000000
3,Astoria Heights,0.000000,0.0,0.000,0.00,0.0,0.000000,0.0,0.00000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000,0.000000,0.000000
4,Auburndale,0.000000,0.0,0.000,0.00,0.0,0.000000,0.0,0.00000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000,0.000000,0.000000
5,Bath Beach,0.000000,0.0,0.000,0.00,0.0,0.000000,0.0,0.00000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.900000,0.000,0.000000,0.000000
6,Battery Park City,0.000000,0.0,0.000,0.00,0.0,0.000000,0.0,0.00000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.800000,0.000,0.050000,0.000000
7,Bay Ridge,0.090909,0.0,0.000,0.00,0.0,0.000000,0.0,0.00000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.727273,0.000,0.000000,0.000000
8,Bay Terrace,0.000000,0.0,0.000,0.00,0.0,0.500000,0.0,0.00000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.500000,0.000,0.000000,0.000000
9,Bayside,0.000000,0.0,0.000,0.00,0.0,0.000000,0.0,0.00000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000,0.000000,0.000000


In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Annadale,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
1,Arden Heights,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
2,Astoria,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar
3,Astoria Heights,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
4,Auburndale,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega


In [32]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 2, 2, 2, 0], dtype=int32)

In [33]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = neighborhoods_dataframe
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bronx,Co-op City,40.874294,-73.829939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bronx,Eastchester,40.887556,-73.827806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bronx,Fieldston,40.895437,-73.905643,0.0,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
4,Bronx,Riverdale,40.890834,-73.912585,0.0,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega


In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
# rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
4,Riverdale,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
5,Kingsbridge,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
6,Marble Hill,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
7,Woodlawn,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
8,Norwood,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
9,Williamsbridge,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
11,Pelham Parkway,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
14,University Heights,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
19,High Bridge,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega


In [39]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
142,Maspeth,Japanese Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar
182,Holliswood,Japanese Restaurant,Vegetarian / Vegan Restaurant,Theme Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
191,Rockaway Park,Japanese Restaurant,Vegetarian / Vegan Restaurant,Theme Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
239,Charleston,Japanese Restaurant,Vegetarian / Vegan Restaurant,Theme Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
300,Erasmus,Asian Restaurant,Japanese Restaurant,Theme Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega


In [40]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Country Club,Sushi Restaurant,Asian Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
37,Pelham Bay,Sushi Restaurant,Asian Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
38,Schuylerville,Sushi Restaurant,Chinese Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Cocktail Bar,Deli / Bodega
46,Bay Ridge,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar
48,Sunset Park,Asian Restaurant,Sushi Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Deli / Bodega
52,Sheepshead Bay,Sushi Restaurant,Poke Place,Japanese Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant
53,Manhattan Terrace,Sushi Restaurant,Japanese Restaurant,Café,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Chinese Restaurant,Cocktail Bar
55,Crown Heights,Sushi Restaurant,Burger Joint,Chinese Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Café,Cocktail Bar,Deli / Bodega
59,Prospect Heights,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Korean Restaurant,Bakery,Bubble Tea Shop,Burger Joint,Café,Cocktail Bar
61,Williamsburg,Sushi Restaurant,Bubble Tea Shop,Japanese Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Bakery,Burger Joint,Café,Chinese Restaurant,Cocktail Bar
